## Quaternion PyTorch - Basic mechanisms

In [1]:
import torch
from htorch import quaternion

### 1 - Quaternion tensors

A quaternion number is represented by:

$$
x = a + bi + cj + dk
$$

where $a$, $b$, $c$, and $d$ are real values, and $i$, $j$, $k$ are the imaginary parts. A `QuaternionTensor` extends the standard PyTorch `tensor` to handle quaternion values, by specifying the real and imaginary components during initialization:

In [2]:
# Simple scalar quaternion
x = quaternion.QuaternionTensor([0.0, 0.3, 0.4, 0.5])
x

real part: tensor([0.])
imaginary part (i): tensor([0.3000])
imaginary part (j): tensor([0.4000])
imaginary part (k): tensor([0.5000])

In [3]:
# Mini-batch of two scalar quaternions
x = quaternion.QuaternionTensor(torch.rand(2, 4))
print(x)

real part: tensor([[0.1914],
        [0.8278]])
imaginary part (i): tensor([[0.2804],
        [0.5887]])
imaginary part (j): tensor([[0.9123],
        [0.6023]])
imaginary part (k): tensor([[0.9083],
        [0.7434]])


In [4]:
# A vector with 4 quaternions
y = quaternion.QuaternionTensor(torch.rand(16))
print(y)

real part: tensor([0.1624, 0.1938, 0.4980, 0.8163])
imaginary part (i): tensor([0.0568, 0.2769, 0.6222, 0.7837])
imaginary part (j): tensor([0.7847, 0.3980, 0.9395, 0.1019])
imaginary part (k): tensor([0.9743, 0.8304, 0.0602, 0.2409])


All standard quaternion operations can be applied on the tensor (see `QuaternionTensor` for a full list):

In [5]:
# Conjugation
print(x.conj())

real part: tensor([[0.1914],
        [0.8278]])
imaginary part (i): tensor([[-0.2804],
        [-0.5887]])
imaginary part (j): tensor([[-0.9123],
        [-0.6023]])
imaginary part (k): tensor([[-0.9083],
        [-0.7434]])


In [6]:
# Element-wise norm
print(x.norm())

tensor([[1.3314],
        [1.3955]])


In [7]:
# Element-wise angle
print(x.theta())

tensor([[1.4265],
        [0.9357]])


In [8]:
# Quaternion multiplication (Hamilton product)
print(x * x)

real part: tensor([[-1.6992],
        [-0.5767]])
imaginary part (i): tensor([[0.1074],
        [0.9747]])
imaginary part (j): tensor([[0.3493],
        [0.9972]])
imaginary part (k): tensor([[0.3478],
        [1.2308]])


In [9]:
# Quaternion matrix multiplication
print(x.t() @ x)

tensor([[1.7725, 1.5482],
        [1.5482, 1.9473]])


Importantly, quaternion tensors and real-valued tensors are interoperable (real-valued tensors being casted to quaternion tensors with 0 imaginary parts):

In [10]:
# Quaternion scalar multiplication
print(x * torch.rand(2))

real part: tensor([[0.1737],
        [0.4250]])
imaginary part (i): tensor([[0.2545],
        [0.3023]])
imaginary part (j): tensor([[0.8280],
        [0.3092]])
imaginary part (k): tensor([[0.8244],
        [0.3817]])


### 2 - Quaternion gradients

Gradients can be computed with the PyTorch autograd mechanisms:

In [11]:
x = quaternion.QuaternionTensor(torch.rand(2, 4))
x.requires_grad = True
y = x.norm().sum()
y.backward()

### 3 - Quaternion-valued layers

We also provide a number of quaternion-valued layers to implement quaternion neural networks:

In [12]:
from torch import nn
from htorch.layers import QLinear

In [13]:
model = nn.Sequential(
    QLinear(4, 20),
    nn.ReLU(),
    QLinear(20, 1)
)

In [14]:
x = quaternion.QuaternionTensor(torch.rand(2, 16))
print(model(x))

real part: tensor([[ 0.1307],
        [-0.3902]], grad_fn=<SliceBackward>)
imaginary part (i): tensor([[ 0.1433],
        [-0.0184]], grad_fn=<SliceBackward>)
imaginary part (j): tensor([[0.1187],
        [0.0762]], grad_fn=<SliceBackward>)
imaginary part (k): tensor([[-0.2229],
        [-0.2515]], grad_fn=<SliceBackward>)


In [15]:
model(x).shape

torch.Size([2, 4])

We also provide layers to easily integrate quaternion-valued and real-valued blocks:

In [17]:
from htorch.layers import QuaternionToReal
from torch.nn import Softmax

In [22]:
model = nn.Sequential(
    QLinear(4, 10),
    QuaternionToReal(10), # Take the absolute value
    Softmax(1)
)

In [23]:
model(x).shape

torch.Size([2, 10])